In [19]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [21]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [22]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade,Maranhão - Consumo de Cimento (t)
0,2014,11.986521,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.664236,5.676413e+08,2.779340e+09,1.954361e+10,...,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.593989,0.738420,1303.411571
1,2015,12.529442,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.677816,5.179392e+08,2.561731e+09,1.327596e+10,...,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.607501,0.747557,1191.509657
2,2016,12.762358,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.678618,4.524144e+08,2.516859e+09,1.011370e+10,...,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.607001,0.746972,1100.217771
3,2017,12.460135,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.678492,4.213689e+08,2.497710e+09,8.456309e+09,...,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.606273,0.745878,971.921000
4,2018,12.318837,5.213468e+07,2.563399e+06,6.219897,4.066060e+07,0.677901,4.166580e+08,2.797145e+09,7.937760e+09,...,27.340044,1005.849486,1408.786927,26.817246,23.337177,82.235372,1.701278,0.605389,0.744231,750.012000
5,2019,12.204421,5.168108e+07,2.580537e+06,6.200907,4.050429e+07,0.677164,4.462625e+08,3.484252e+09,1.091282e+10,...,26.926167,1005.716797,1523.397634,26.859978,23.384893,82.273174,1.691383,0.604697,0.741979,774.045000
6,2020,12.039300,5.119067e+07,2.602928e+06,6.197992,4.043941e+07,0.675634,5.489412e+08,4.114911e+09,1.625333e+10,...,27.098804,1005.562554,1497.079893,26.790079,23.352799,82.421548,1.721091,0.603852,0.739159,1038.249000
7,2021,11.842073,5.015174e+07,2.613053e+06,6.179648,4.024163e+07,0.673510,6.638811e+08,4.421848e+09,1.933031e+10,...,-25.103030,1005.494631,1473.536669,26.708086,23.251450,82.286140,1.737255,0.602293,0.737446,1059.129000
8,2022,11.856801,4.889502e+07,2.622836e+06,6.156726,3.999469e+07,0.673283,7.010332e+08,4.533216e+09,1.997888e+10,...,-197.895668,1005.486775,1454.853030,26.696056,23.172583,81.955870,1.762953,0.602228,0.739486,1083.999000


In [23]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Maranhão - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade
0,11.986521,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.664236,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,687.203882,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.593989,0.738420
1,12.529442,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.677816,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,630.565434,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.607501,0.747557
2,12.762358,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.678618,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,538.170446,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.607001,0.746972
3,12.460135,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.678492,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,440.490154,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.606273,0.745878
4,12.318837,5.213468e+07,2.563399e+06,6.219897,4.066060e+07,0.677901,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,379.224267,27.340044,1005.849486,1408.786927,26.817246,23.337177,82.235372,1.701278,0.605389,0.744231
5,12.204421,5.168108e+07,2.580537e+06,6.200907,4.050429e+07,0.677164,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,358.628076,26.926167,1005.716797,1523.397634,26.859978,23.384893,82.273174,1.691383,0.604697,0.741979
6,12.039300,5.119067e+07,2.602928e+06,6.197992,4.043941e+07,0.675634,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,359.150605,27.098804,1005.562554,1497.079893,26.790079,23.352799,82.421548,1.721091,0.603852,0.739159


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    1191.509657
1    1100.217771
2     971.921000
3     750.012000
4     774.045000
5    1038.249000
6    1059.129000
7    1083.999000
8            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [25]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Maranhão - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade
0,-1.584514,-1.735560,2.001412,1.096595,0.562643,-2.226454,1.779974,0.019134,2.003355,-1.880579,...,1.469922,0.477541,-0.910793,0.760740,0.554524,-0.802094,-0.822066,1.360743,-2.190080,-1.817562
1,0.618861,-0.375980,0.468217,0.040184,0.182001,0.409975,0.870373,-0.621387,0.401153,0.111653,...,1.011197,1.238330,-0.049168,-0.282914,1.420717,0.344933,-1.031985,1.052539,0.724550,1.069326
2,1.564117,0.751470,-0.564635,0.333595,0.650140,0.565576,-0.328802,-0.753466,-0.407217,1.154682,...,0.262873,0.972189,-0.925541,-1.055320,0.844163,-0.568140,-0.837183,0.302713,0.616709,0.884277
3,0.337585,1.097882,-0.893594,0.428396,0.665197,0.541184,-0.896969,-0.809829,-0.830898,0.955301,...,-0.528257,-0.190602,0.795763,-1.270193,-0.561713,-1.273468,0.024590,-0.335950,0.459845,0.538795
4,-0.235855,0.852914,-0.776193,0.212164,0.123998,0.426331,-0.983185,0.071542,-0.963455,0.100493,...,-1.024461,-0.985702,1.786725,0.250355,-1.349468,0.578861,1.299846,-1.146804,0.269111,0.018296
5,-0.700194,-0.590727,-0.235207,-2.110934,-2.183979,0.283388,-0.441390,2.094006,-0.202939,-0.441549,...,-1.191273,-1.511756,-0.696985,1.597331,-0.908223,1.719907,1.366797,-1.233241,0.119866,-0.693132


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    1191.509657
1    1100.217771
2     971.921000
3     750.012000
4     774.045000
5    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [27]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Maranhão - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade
6,-1.126536,-1.545437,0.429838,-1.670629,-1.873698,-0.012722,1.16956,2.035192,0.985175,-0.705937,...,-1.002692,-1.075068,-1.970714,1.072178,-1.284821,0.829662,1.28459,-0.846021,-0.0579,-1.258308


In [28]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [29]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4096, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [30]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [31]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2939964741, 913308463, 2725693925, 865128266, 682056969, 3797039038, 3686690255, 3061870509, 3379315913, 2501794896]


Step: 0 ___________________________________________
Epoch 1922: early stopping
val_loss: 33911.1875
winner_seed: 2939964741


Step: 1 ___________________________________________
Epoch 1910: early stopping
val_loss: 1621.2369384765625
winner_seed: 913308463


Step: 2 ___________________________________________
Epoch 1752: early stopping
val_loss: 33911.2109375


Step: 3 ___________________________________________
Epoch 759: early stopping
val_loss: 0.006451666355133057
winner_seed: 865128266


Step: 4 ___________________________________________
Epoch 1751: early stopping
val_loss: 1367.825927734375


Step: 5 ___________________________________________
val_loss: 0.0004155784845352173
winner_seed: 3797039038


Step: 6 ___________________________________________
Epoch 1754: early stopping
val_loss: 1367.825927734375


Step: 7 ___________________________________________
Ep

In [32]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 1s 551ms/step - loss: 969096.5625 - val_loss: 1065903.5000
Epoch 2/5000
1/1 [==============================] - 0s 51ms/step - loss: 940374.8125 - val_loss: 1523344.3750
Epoch 3/5000
1/1 [==============================] - 0s 50ms/step - loss: 881290.1875 - val_loss: 905141.1250
Epoch 4/5000
1/1 [==============================] - 0s 51ms/step - loss: 936336.0625 - val_loss: 943727.6875
Epoch 5/5000
1/1 [==============================] - 0s 54ms/step - loss: 901358.6875 - val_loss: 1069799.3750
Epoch 6/5000
1/1 [==============================] - 0s 51ms/step - loss: 855292.5625 - val_loss: 977178.9375
Epoch 7/5000
1/1 [==============================] - 0s 52ms/step - loss: 846093.8125 - val_loss: 928737.1250
Epoch 8/5000
1/1 [==============================] - 0s 51ms/step - loss: 851394.5625 - val_loss: 1231412.2500
Epoch 9/5000
1/1 [==============================] - 0s 53ms/step - loss: 782660.8125 - val_loss: 1314137.0000
Epoch 10/5000

1/1 [==============================] - 0s 53ms/step - loss: 190775.9844 - val_loss: 522948.5312
Epoch 76/5000
1/1 [==============================] - 0s 56ms/step - loss: 188421.5156 - val_loss: 518370.1250
Epoch 77/5000
1/1 [==============================] - 0s 51ms/step - loss: 186109.8281 - val_loss: 513871.0938
Epoch 78/5000
1/1 [==============================] - 0s 58ms/step - loss: 183839.7969 - val_loss: 509448.9062
Epoch 79/5000
1/1 [==============================] - 0s 52ms/step - loss: 181610.0781 - val_loss: 505101.2812
Epoch 80/5000
1/1 [==============================] - 0s 55ms/step - loss: 179419.6719 - val_loss: 500825.7500
Epoch 81/5000
1/1 [==============================] - 0s 53ms/step - loss: 177267.4219 - val_loss: 496620.0938
Epoch 82/5000
1/1 [==============================] - 0s 54ms/step - loss: 175152.3750 - val_loss: 492482.2812
Epoch 83/5000
1/1 [==============================] - 0s 52ms/step - loss: 173073.5312 - val_loss: 488410.5625
Epoch 84/5000
1/1 [=====

1/1 [==============================] - 0s 59ms/step - loss: 87620.4766 - val_loss: 307053.5625
Epoch 150/5000
1/1 [==============================] - 0s 57ms/step - loss: 86842.8672 - val_loss: 305129.3438
Epoch 151/5000
1/1 [==============================] - 0s 58ms/step - loss: 86075.4062 - val_loss: 303221.2188
Epoch 152/5000
1/1 [==============================] - 0s 54ms/step - loss: 85317.9297 - val_loss: 301329.0312
Epoch 153/5000
1/1 [==============================] - 0s 56ms/step - loss: 84570.3047 - val_loss: 299452.5000
Epoch 154/5000
1/1 [==============================] - 0s 55ms/step - loss: 83832.3203 - val_loss: 297591.4375
Epoch 155/5000
1/1 [==============================] - 0s 54ms/step - loss: 83103.8672 - val_loss: 295745.7188
Epoch 156/5000
1/1 [==============================] - 0s 53ms/step - loss: 82384.8125 - val_loss: 293915.1250
Epoch 157/5000
1/1 [==============================] - 0s 53ms/step - loss: 81674.9531 - val_loss: 292099.3438
Epoch 158/5000
1/1 [=====

1/1 [==============================] - 0s 53ms/step - loss: 49591.2344 - val_loss: 198177.9375
Epoch 224/5000
1/1 [==============================] - 0s 52ms/step - loss: 49264.4102 - val_loss: 197067.0625
Epoch 225/5000
1/1 [==============================] - 0s 53ms/step - loss: 48940.9062 - val_loss: 195963.5781
Epoch 226/5000
1/1 [==============================] - 0s 48ms/step - loss: 48620.6719 - val_loss: 194867.6094
Epoch 227/5000
1/1 [==============================] - 0s 52ms/step - loss: 48303.6719 - val_loss: 193778.9062
Epoch 228/5000
1/1 [==============================] - 0s 52ms/step - loss: 47989.8398 - val_loss: 192697.4219
Epoch 229/5000
1/1 [==============================] - 0s 52ms/step - loss: 47679.1523 - val_loss: 191623.1406
Epoch 230/5000
1/1 [==============================] - 0s 51ms/step - loss: 47371.5586 - val_loss: 190556.1719
Epoch 231/5000
1/1 [==============================] - 0s 53ms/step - loss: 47067.0273 - val_loss: 189496.2344
Epoch 232/5000
1/1 [=====

1/1 [==============================] - 0s 54ms/step - loss: 32052.3516 - val_loss: 132678.8594
Epoch 298/5000
1/1 [==============================] - 0s 54ms/step - loss: 31882.5527 - val_loss: 131984.7031
Epoch 299/5000
1/1 [==============================] - 0s 54ms/step - loss: 31714.0410 - val_loss: 131294.5781
Epoch 300/5000
1/1 [==============================] - 0s 54ms/step - loss: 31546.7988 - val_loss: 130608.6953
Epoch 301/5000
1/1 [==============================] - 0s 51ms/step - loss: 31380.8145 - val_loss: 129926.8984
Epoch 302/5000
1/1 [==============================] - 0s 50ms/step - loss: 31216.0684 - val_loss: 129249.1562
Epoch 303/5000
1/1 [==============================] - 0s 54ms/step - loss: 31052.5469 - val_loss: 128575.4297
Epoch 304/5000
1/1 [==============================] - 0s 51ms/step - loss: 30890.2363 - val_loss: 127905.7656
Epoch 305/5000
1/1 [==============================] - 0s 54ms/step - loss: 30729.1309 - val_loss: 127240.0391
Epoch 306/5000
1/1 [=====

...

1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 5.9605e-04
Epoch 4683/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 5.9605e-04
Epoch 4684/5000
1/1 [==============================] - 0s 51ms/step - loss: 1371.6715 - val_loss: 5.9605e-04
Epoch 4685/5000
1/1 [==============================] - 0s 72ms/step - loss: 1371.6715 - val_loss: 5.9605e-04
Epoch 4686/5000
1/1 [==============================] - 0s 68ms/step - loss: 1371.6715 - val_loss: 5.9605e-04
Epoch 4687/5000
1/1 [==============================] - 0s 57ms/step - loss: 1371.6715 - val_loss: 5.9010e-04
Epoch 4688/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 5.9010e-04
Epoch 4689/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6715 - val_loss: 5.9010e-04
Epoch 4690/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6718 - val_loss: 5.9010e-04
Epoch 4691/5000
1/1 [==============

1/1 [==============================] - 0s 69ms/step - loss: 1371.6715 - val_loss: 5.4361e-04
Epoch 4758/5000
1/1 [==============================] - 0s 64ms/step - loss: 1371.6718 - val_loss: 5.4361e-04
Epoch 4759/5000
1/1 [==============================] - 0s 53ms/step - loss: 1371.6718 - val_loss: 5.4361e-04
Epoch 4760/5000
1/1 [==============================] - 0s 53ms/step - loss: 1371.6718 - val_loss: 5.4361e-04
Epoch 4761/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6718 - val_loss: 5.3793e-04
Epoch 4762/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6718 - val_loss: 5.4361e-04
Epoch 4763/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 5.3793e-04
Epoch 4764/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6715 - val_loss: 5.3228e-04
Epoch 4765/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6715 - val_loss: 5.3228e-04
Epoch 4766/5000
1/1 [==============

1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4833/5000
1/1 [==============================] - 0s 58ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4834/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4835/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4836/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4837/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4838/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4839/5000
1/1 [==============================] - 0s 63ms/step - loss: 1371.6715 - val_loss: 4.8818e-04
Epoch 4840/5000
1/1 [==============================] - 0s 75ms/step - loss: 1371.6715 - val_loss: 4.8280e-04
Epoch 4841/5000
1/1 [==============

1/1 [==============================] - 0s 53ms/step - loss: 1371.6715 - val_loss: 4.4084e-04
Epoch 4908/5000
1/1 [==============================] - 0s 57ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4909/5000
1/1 [==============================] - 0s 56ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4910/5000
1/1 [==============================] - 0s 63ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4911/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4912/5000
1/1 [==============================] - 0s 50ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4913/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4914/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4915/5000
1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 4.3572e-04
Epoch 4916/5000
1/1 [==============

1/1 [==============================] - 0s 54ms/step - loss: 1371.6715 - val_loss: 4.1558e-04
Epoch 4983/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4984/5000
1/1 [==============================] - 0s 61ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4985/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4986/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4987/5000
1/1 [==============================] - 0s 53ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4988/5000
1/1 [==============================] - 0s 55ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4989/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.2057e-04
Epoch 4990/5000
1/1 [==============================] - 0s 52ms/step - loss: 1371.6715 - val_loss: 4.1558e-04
Epoch 4991/5000
1/1 [==============

In [33]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 51ms/step
[1059.129](test_target) - [[1038.2286]](prediction) = 20.900362304687405


In [34]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [35]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Maranhão - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade
0,-1.352351,-1.261020,1.296160,1.362076,0.480824,-1.412474,1.165029,1.396038,1.335266,-1.330895,...,1.116140,-1.327552,-0.689032,1.280967,-1.070124,-0.930189,0.784701,1.024998,-1.413461,-1.411894
1,0.317903,0.076098,-0.158206,-1.010525,-1.392196,0.645506,0.111769,-0.502296,-0.264151,0.251276,...,0.194043,1.085919,1.414061,-0.121502,1.335765,1.387624,-1.411263,0.331322,0.746693,0.776071
2,1.034448,1.184921,-1.137954,-0.351551,0.911372,0.766968,-1.276798,-0.893743,-1.071116,1.079619,...,-1.310183,0.241633,-0.725028,-1.159465,-0.265641,-0.457436,0.626562,-1.356320,0.666768,0.635823


train_target:


,Maranhão - Consumo de Cimento (t)
0,1191.509657
1,1100.217771
2,971.921000


test_input:


...

,Maranhão - Desemprego,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Maranhão - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH Renda,Maranhão - IDH Longevidade
6,-1.126536,-1.545437,0.429838,-1.670629,-1.873698,-0.012722,1.16956,2.035192,0.985175,-0.705937,...,-1.002692,-1.075068,-1.970714,1.072178,-1.284821,0.829662,1.28459,-0.846021,-0.0579,-1.258308


test_target:


,Maranhão - Consumo de Cimento (t)
6,1059.129


1/1 [==============================] - 0s 33ms/step
Error: 20.900362304687405




[971.911865234375, 750.0061645507812, 762.0248413085938, 1038.2286376953125]

In [36]:
display(targets)
display(predictions)

[750.012, 774.045, 1038.249, 1059.129]

[971.911865234375, 750.0061645507812, 762.0248413085938, 1038.2286376953125]

In [37]:
mae = mean_absolute_error(predictions, targets)
mae

135.76580541992186

In [38]:
porcentage = mae/np.mean(targets)
porcentage

0.14995801986772855